#  Movies Recomendation by Similarity

In [1]:
# Import Libraries and Resources

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

movies = pd.read_csv('tmdb_5000_movies.csv')

print(movies.shape)


(4803, 20)


In [71]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [2]:
movies_df = movies[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]


### Data Preprocessing


In [3]:
# The complex genre column in string form
movies_df['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [4]:
type(movies_df['genres'][0])


str

In [5]:
# Change to list form to make it easier to extract name values

from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [6]:
movies_df['genres'][0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [7]:
type(movies_df['genres'][0])

list

### Dictionary in list, convert the value of the name key to a list


In [8]:
movies_df['genres']


0       [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
1       [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2       [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
3       [{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...
4       [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...
                              ...                        
4798    [{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...
4799    [{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...
4800    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4801                                                   []
4802                  [{'id': 99, 'name': 'Documentary'}]
Name: genres, Length: 4803, dtype: object

In [9]:
movies_df['genres'][0][2]['name']

'Fantasy'

In [10]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x :[ y['name'] for y in x])

In [11]:
movies_df[['genres']][0:3]

,genres
0,"[Action, Adventure, Fantasy, Science Fiction]"
1,"[Adventure, Fantasy, Action]"
2,"[Action, Adventure, Crime]"


## Genre similarity measurement


In [12]:
from sklearn.feature_extraction.text import CountVectorizer


In [13]:
# To apply CountVectorizer, change the genres information to a space-separated string and apply it to a new column (using join)

movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
movies_df['genres_literal'][0:3]

0    Action Adventure Fantasy Science Fiction
1                    Adventure Fantasy Action
2                      Action Adventure Crime
Name: genres_literal, dtype: object

In [14]:
type(movies_df['genres_literal'][0])

str

In [15]:
# CountVectorizer 

# Range of minimum frequency(min_df) and ngram 
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2)) 

# Index dictionary by genre
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


In [17]:
# Narrowed CountVectorizer learning

count_vect2 = CountVectorizer(min_df=1, ngram_range=(1, 1))
genre_mat2 = count_vect2.fit_transform(movies_df['genres_literal'])
print(genre_mat2.shape)

(4803, 22)


### Calculate the cosine similarity

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
genre_sim = cosine_similarity(genre_mat, genre_mat)

print(genre_sim.shape)
print(genre_sim[::-1])

(4803, 4803)
[[0.         0.         0.         ... 0.         0.         1.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 1.         0.         0.        ]
 ...
 [0.4472136  0.4        1.         ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]
 [1.         0.59628479 0.4472136  ... 0.         0.         0.        ]]


In [19]:
# Display movies in order of similarity

genre_sim_sorted_ind = genre_sim.argsort()[:,::-1]
genre_sim_sorted_ind[::-1]

array([[4802, 4710, 4521, ..., 3140, 3141,    0],
       [4802, 1594, 1596, ..., 3204, 3205,    0],
       [4800, 3809, 1895, ..., 2229, 2230,    0],
       ...,
       [   2, 1740, 1542, ..., 3000, 2999, 2401],
       [ 262,    1,  129, ..., 3069, 3067, 2401],
       [   0, 3494,  813, ..., 3038, 3037, 2401]], dtype=int64)

### Movie recommendation using similarity of genre cosine.


In [51]:
title_movie = movies_df[movies_df['title'] == 'The Godfather']
title_movie

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal,weighted_vote
3337,238,The Godfather,"[Drama, Crime]",8.4,5893,143.659698,"[italy, love at first sight, loss of father, p...","Spanning the years 1945 to 1955, a chronicle o...",Drama Crime,8.263591


In [96]:
# Store movie indexes as variables
title_index = title_movie.index.values

# Extract 10 indexes of movies with similar cosine similarity
similar_indexes = genre_sim_sorted_ind[title_index, :10]
similar_indexes = similar_indexes.reshape(-1)

# Change array
movies_df.iloc[similar_indexes].head(1)

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal,weighted_vote
2731,240,The Godfather: Part II,"[Drama, Crime]",8.3,3338,105.792936,"[italo-american, cuba, vororte, melancholy, pr...",In the continuing saga of the Corleone crime f...,Drama Crime,8.079586


In [97]:
def find_sim_movie_ver1(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
        
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    similar_indexes = similar_indexes.reshape(-1)
    return df.iloc[similar_indexes]

### 10 movies similar to The Godfather recommended


In [95]:
similar_movies = find_sim_movie_ver1(movies_df, genre_sim_sorted_ind, 'The Godfather', 20)
similar_movies[['title', 'genres',]].head(10)

,title,genres
2731,The Godfather: Part II,"[Drama, Crime]"
1243,Mean Streets,"[Drama, Crime]"
3636,Light Sleeper,"[Drama, Crime]"
1946,The Bad Lieutenant: Port of Call - New Orleans,"[Drama, Crime]"
2640,Things to Do in Denver When You're Dead,"[Drama, Crime]"
4065,Mi America,"[Drama, Crime]"
1847,GoodFellas,"[Drama, Crime]"
4217,Kids,"[Drama, Crime]"
883,Catch Me If You Can,"[Drama, Crime]"
3866,City of God,"[Drama, Crime]"


In [92]:
movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]


,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


### Reflect  vote scores on recommendation system

v: Number of times a movie's rating was voted (vote_count) #Variable value
m: Criteria to be rated (minimum number of votes) -> Top 60% of votes here
R: Ratings of individual films #Variable values
C: Ratings for the entire film

(v/(v+m))*R + (m/(v+m))*C  ---> Formula

In [56]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)

In [57]:
print('C:', round(C,3), 'm:',round(m,3))

C: 6.092 m: 370.2


In [58]:
def weighted_vote_average(record): 
    v = record['vote_count']
    R = record['vote_average']
    
    return ((v/(v+m))*R)+((m/(m+v))*C)

In [59]:
movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis = 1)

In [87]:
movies_df.head(1)

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal,weighted_vote
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction,7.166301


In [61]:
movies_df[['weighted_vote','title','vote_average','vote_count','genres']].sort_values('weighted_vote',ascending=False)[:10]


,weighted_vote,title,vote_average,vote_count,genres
1881,8.396052,The Shawshank Redemption,8.5,8205,"[Drama, Crime]"
3337,8.263591,The Godfather,8.4,5893,"[Drama, Crime]"
662,8.216455,Fight Club,8.3,9413,[Drama]
3232,8.207102,Pulp Fiction,8.3,8428,"[Thriller, Crime]"
65,8.136930,The Dark Knight,8.2,12002,"[Drama, Action, Crime, Thriller]"
1818,8.126069,Schindler's List,8.3,4329,"[Drama, History, War]"
3865,8.123248,Whiplash,8.3,4254,[Drama]
809,8.105954,Forrest Gump,8.2,7927,"[Comedy, Drama, Romance]"
2294,8.105867,Spirited Away,8.3,3840,"[Fantasy, Adventure, Animation, Family]"
2731,8.079586,The Godfather: Part II,8.3,3338,"[Drama, Crime]"
